In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.insert(0, "/home/srs-9/Projects/ms_mri/analysis/thalamus/helpers")

import helpers
import utils
import json

In [ ]:
max_dzdur = None

hips_thomas_ref = pd.read_csv(
    "/home/srs-9/Projects/ms_mri/data/hipsthomas_struct_index.csv", index_col="index"
)["struct"]
thalamic_nuclei = [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
with open("/home/srs-9/Projects/ms_mri/longitudinal_pipeline/all_longitudinal_sessions.json", 'r') as f:
    subject_sessions = json.load(f)
df_long_full = pd.read_csv("/home/srs-9/Projects/ms_mri/longitudinal_pipeline/data0/full_volumes_long.csv").set_index(["subid", "sesid"])
